In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('Qlo_Poster_IE.pdf')
docs=loader.load()

/Users/apple/Desktop/LangChain/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-11-16T14:21:20+00:00', 'title': 'Qlo_Poster_IE', 'moddate': '2025-11-16T14:21:20+00:00', 'keywords': 'DAG43T_Crz8,BAEthX5MxqM,0', 'author': 'Suvansh Sehgal', 'containsaigeneratedcontent': 'Yes', 'source': 'Qlo_Poster_IE.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='P r o d u c t s\na n d  S e r v i c e s\nG a i n  C r e a t o r s\nG a i n s\nP a i n  R e l i e v e r s\nP a i n s\nC u s t o m e r\nJ o b s\nIn s t a n t  a c c e s s  t o  c lo t h e s\nL a s t - m in u t e  o u t f it  s o lu t io n s\nE a s y  s h o p p in g  e x p e r ie n c e\nB e t t e r  s h o p  r e a c h\n3 0 - m in  c lo t h in g\nd e l iv e r y\nT r y - t h e n - B u y\nF a s t  r e t u r n s\nL o c a l  s h o p  l is t in g s\nR e a l- t im e  t r a c k in g\nIn s t a n t  a c c e s s  t o  c lo t h e s\nL a s t - m in u t e  o u t f it  s o lu t io n s\nE a s y  s h o p p in g  e x p e r ie n c e\nB e t t 

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS


embeddings = OllamaEmbeddings(model="phi")
db = FAISS.from_documents(documents[:10],embedding=embeddings)

In [5]:
db

In [6]:
query="What is the problem statement?"
result=db.similarity_search(query)
result[0].page_content

'B U S I N E S S  M O D E L  C A N V A S\nL o c a l  g a r m e n t  s h o p s\n&  b o u t iq u e s\nD e l iv e r y  p a r t n e r s\n( i n d e p e n d e n t  r id e r s )\nP a y m e n t  g a t e w a y\np a r t n e r s  ( R a z o r p a y ,\nP a y t m , e t c .)\nA p p  d e v e l o p m e n t  &\nm a i n t e n a n c e  t e a m\nP a c k a g i n g  s u p p l ie r s\n( f o r  t r y - t h e n - b u y  &\nr e t u r n  b a g s )\n30-minute delivery forclothes from localshopsTry-then-Buy optionfor customersInstantreturns/exchangesHelps shops go digitalwithout investmentFaster than onlinemarketplaces (3–5days →  30 mins)\nR e a l- t i m e  o r d e r  t r a c k i n g\nI n - a p p  c u s t o m e r  s u p p o r t\nE a s y  r e t u r n / e x c h a n g e\np r o c e s s\nP e r s o n a l iz e d  c l o t h i n g\ns u g g e s t io n s  b a s e d  o n\nl o c a t io n\nN o t if ic a t io n s  f o r  o f f e r s  f r o m\nn e a r b y  s h o p s\nS t u d e n t s\nO f f ic e  w o r k e r s\nL a s t - m i n u t

In [7]:
from langchain_community.llms import Ollama
# Load Ollama phi llm model
llm = Ollama(model="phi")
llm

/var/folders/8z/lfg_cf355xj5jgmz32jg4kpm0000gn/T/ipykernel_25708/2108800857.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi")


Ollama(model='phi')

In [8]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>
Question: {input}""")

In [9]:
## Chain Introduction
## create stuff document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)

In [ ]:
# Retriever
retriever = db.as_retriever()

In [11]:
## Retrieval chain
from langchain.chains import create_retrieval_chain
retrieval_chain= create_retrieval_chain(retriever,document_chain)


In [13]:
response=retrieval_chain.invoke({"input":"Online clothing platforms take 3–5 days to deliver,which is inconvenient for people needing outfits urgently."})

In [14]:
response['answer']

' The online clothing platforms are not suitable for individuals who require immediate outfit options. They offer 3-to-5 day delivery times, which may not align with the specific needs of those seeking outfits in a timely manner. Therefore, it would be beneficial for them to explore other options that provide faster delivery options.\n'